In [15]:
import cv2
import numpy as np
t=[[]]
blurValue=7
threshold=10
count=0
x=1000
y=1000
x1=[]
x2=[]
y1=[]
y2=[]
im=[]
l=[]
prediction=[]
import pickle
clf = pickle.load(open('model1.sav', 'rb'))
camera = cv2.VideoCapture(1)
while True:
    ret, image = camera.read()
    cv2.imshow('original', image)
    k = cv2.waitKey(1)
    if k == ord('c'):
        camera.release()
        cv2.destroyAllWindows()
        #Read image
        #Create a dummy image that stores different contrast and brightness
        #new_image = np.zeros(image.shape, image.dtype)
        #Brightness and contrast parameters
        #contrast = 3.0
        #bright = 0
        #Change the contrast and brightness
#         for y2 in range(image.shape[0]):
#             for x2 in range(image.shape[1]):
#                 for c2 in range(image.shape[2]):
#                     new_image[y2,x2,c2] = np.clip(contrast*image[y2,x2,c2] + bright, 0, 255)
#         #image = cv2.resize(image, (28,28), interpolation = cv2.INTER_AREA)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (blurValue, blurValue), 0)
        #ret, thresh = cv2.threshold(blur, threshold, 255, cv2.THRESH_BINARY_INV)
        thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, blockSize=25, C=10)
        #ret2,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            l.append(cv2.contourArea(cnt))
        l.sort(reverse=True)
        for cnt in contours:
            if cv2.contourArea(cnt)<(l[0]/20):
                q,y,w,h = cv2.boundingRect(cnt)
                for i in range(y,y+h):
                    for j in range(q,q+w):
                        thresh[i][j]=0
        for i in range(thresh.shape[1]):
            old=count
            for j in range(thresh.shape[0]):
                if thresh[j][i]==255:
                    count+=1
            if count>old:
                x=min(x,i)
            if count>0 and old==count:
                x2.append(i)
                x1.append(x)
                x=1000
                count=0
        if old!=count:
            x2.append(thresh.shape[1])
            x1.append(x)
        count=0
        for z in range(len(x1)):
            for i in range(thresh.shape[0]):
                old=count
                for j in range(x1[z],x2[z]):
                    if thresh[i][j]==255:
                        count+=1
                if count>old:
                    y=min(y,i)
                if count>0 and old==count:
                    y2.append(i)
                    y1.append(y)
                    y=1000
                    count=0
            if old!=count:
                y2.append(thresh.shape[0])
                y1.append(y)
        for i in range(len(x1)):
            imcrop = thresh[y1[i]:y2[i], x1[i]:x2[i]]
            a=28-y2[i]+y1[i]
            b=28-x2[i]+x1[i]
            a=max(max(a,0),(x2[i]-x1[i])-(y2[i]-y1[i]))
            b=max(max(b,0),(y2[i]-y1[i])-(x2[i]-x1[i]))
            constan = cv2.copyMakeBorder(imcrop,int(a/2),a-int(a/2),int(b/2),b-int(b/2),cv2.BORDER_CONSTANT)
            imag = cv2.resize(constan, (28,28), interpolation = cv2.INTER_AREA)
            ret, thr = cv2.threshold(imag, threshold, 255, cv2.THRESH_BINARY)
            #thr = cv2.adaptiveThreshold(imag, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, blockSize=5, C=5)
            kernel = np.ones((2,2), np.uint8)
            thr = cv2.dilate(thr, kernel, iterations=1)
            for l in range(28):
                for j in range(28):
                    t[0].append(int(thr[l][j]/255))
            predicted = clf.predict(t)
            prediction.append(predicted[0])
            print(predicted)
            t=[[]]
        break
    elif k == 27:  # press ESC to exit
        break
cv2.destroyAllWindows()

[1]
[8]
[13]
[1]
[2]


In [11]:
j=[]
d=[]
i=0
s=''
while i < len(prediction):
    c=0
    while i<len(prediction) and prediction[i]<=9:
        c=c*10+prediction[i]
        i+=1
    d.append(c)
    s+=str(c)
    if i<len(prediction):
        j.append(prediction[i])
        if prediction[i]==10:
            s+=' / '
        if prediction[i]==11:
            s+=' * '
        if prediction[i]==12:
            s+=' + '
        if prediction[i]==13:
            s+=' - '
    i+=1

ans = d[0]
for i in range(0,len(j)):
    if j[i]==10:
        ans = ans/d[i+1]
    if j[i]==11:
        ans = ans*d[i+1]
    if j[i]==12:
        ans = ans+d[i+1]
    if j[i]==13:
        ans = ans-d[i+1]
print(ans)
s+=' = '+str(ans)    
    
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(image,s,(30,450), font, 2,(255,255,255),2,cv2.LINE_AA)

cv2.imshow('original', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

6


In [4]:
print(x1,x2,y1,y2)

[118] [274] [73] [354]


In [85]:
imcrop=thresh[y1[0]:y2[0], x1[0]:x2[0]]
a=28-y2[0]+y1[0]
b=28-x2[0]+x1[0]
a=max(max(a,0),(x2[0]-x1[0])-(y2[0]-y1[0]))
b=max(max(b,0),(y2[0]-y1[0])-(x2[0]-x1[0]))
constant= cv2.copyMakeBorder(imcrop,int(a/2),a-int(a/2),int(b/2),b-int(b/2),cv2.BORDER_CONSTANT)
image = cv2.resize(constant, (28,28), interpolation = cv2.INTER_AREA)
ret, thr = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)

In [16]:
cv2.imshow('original', thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
j=[]
d=[]
i=0
s=''
while i < len(prediction):
    c=0
    while i<len(prediction) and prediction[i]<=9:
        c=c*10+prediction[i]
        i+=1
    d.append(c)
    s+=str(c)
    if i<len(prediction):
        j.append(prediction[i])
        if prediction[i]==10:
            s+=' / '
        if prediction[i]==11:
            s+=' * '
        if prediction[i]==12:
            s+=' + '
        if prediction[i]==13:
            s+=' - '
    i+=1

In [42]:
ans = d[0]
for i in range(0,len(j)):
    if j[i]==10:
        ans = ans/d[i+1]
    if j[i]==11:
        ans = ans*d[i+1]
    if j[i]==12:
        ans = ans+d[i+1]
    if j[i]==13:
        ans = ans-d[i+1]
print(ans)
s+=' = '+str(ans)

4


In [21]:
import pickle
count=0
clf = pickle.load(open('model1.sav', 'rb'))
for image in im:
    for l in range(28):
        for j in range(28):
            t[count].append(int(image[l][j]/255))
    predicted = clf.predict(t)
    print(predicted)
    count+=1

ValueError: shapes (1,3920) and (784,150) not aligned: 3920 (dim 1) != 784 (dim 0)

In [12]:
for cnt in contours:
    print(cv2.contourArea(cnt))

1.5
2.0
0.0
15.5
17.5
249.0
17.0
2629.5
1462.5
7.0
364.5
6076.5
2763.5
2078.5
511.0
0.0
6.5
0.0
0.0
